In [13]:
import numpy as np
import pandas as pd
import itertools
import pickle
import sys
import os
import dsc
from dsc.query_engine import Query_Processor as dscQP
from dsc import dsc_io

In [14]:
dsc_output = "/gpfs/commons/groups/knowles_lab/sbanerjee/sparse-regression/gradvi-experiments/linreg_indep"
#dsc_output = "/gpfs/commons/groups/knowles_lab/sbanerjee/sparse-regression/gradvi-experiments/linreg_blockdiag"
dsc_fname  = os.path.basename(os.path.normpath(dsc_output))
db = os.path.join(dsc_output, dsc_fname + ".db")
respkl   = os.path.join("../dsc/results", dsc_fname + ".pkl")
dscoutpkl = os.path.join("../dsc/results", dsc_fname + "_dscout.pkl")

if os.path.isfile(dscoutpkl):
    dscout = pd.read_pickle(dscoutpkl)
else:
    print ("Could not find dscout from dscquery")

refresh_pickle = True

target = ["simulate", "simulate.sfix", "simulate.pve", "simulate.se", "simulate.dims", "fit"]
#condition = ["simulate.sfix == 2", "simulate.signal == 'normal'", "simulate.dims == '(50, 200)'"]
#groups = ["fit_cpt:"]
condition = [""]

print ("Reading from DSC database:")
print (f"    {db}")
print ("")

qp = dscQP(db, target, condition)
df = qp.output_table

df

Reading from DSC database:
    /gpfs/commons/groups/knowles_lab/sbanerjee/sparse-regression/gradvi-experiments/linreg_indep/linreg_indep.db



,DSC,simulate,simulate.sfix,simulate.se:output,simulate.pve,simulate.output.file,simulate.dims,fit,fit.output.file
0,1,equicorrgauss,5,equicorrgauss/equicorrgauss_1,0.4,equicorrgauss/equicorrgauss_1,"(500,10000)",mr_ash,mr_ash/equicorrgauss_1_mr_ash_1
1,1,equicorrgauss,10,equicorrgauss/equicorrgauss_2,0.4,equicorrgauss/equicorrgauss_2,"(500,10000)",mr_ash,mr_ash/equicorrgauss_2_mr_ash_1
2,1,equicorrgauss,20,equicorrgauss/equicorrgauss_3,0.4,equicorrgauss/equicorrgauss_3,"(500,10000)",mr_ash,mr_ash/equicorrgauss_3_mr_ash_1
3,1,equicorrgauss,250,equicorrgauss/equicorrgauss_4,0.4,equicorrgauss/equicorrgauss_4,"(500,10000)",mr_ash,mr_ash/equicorrgauss_4_mr_ash_1
4,1,equicorrgauss,5,equicorrgauss/equicorrgauss_5,0.6,equicorrgauss/equicorrgauss_5,"(500,10000)",mr_ash,mr_ash/equicorrgauss_5_mr_ash_1
...,...,...,...,...,...,...,...,...,...
715,10,equicorrgauss,250,equicorrgauss/equicorrgauss_116,0.6,equicorrgauss/equicorrgauss_116,"(500,10000)",gradvi_compound_lasso_init,gradvi_compound_lasso_init/equicorrgauss_116_l...
716,10,equicorrgauss,5,equicorrgauss/equicorrgauss_117,0.8,equicorrgauss/equicorrgauss_117,"(500,10000)",gradvi_compound_lasso_init,gradvi_compound_lasso_init/equicorrgauss_117_l...
717,10,equicorrgauss,10,equicorrgauss/equicorrgauss_118,0.8,equicorrgauss/equicorrgauss_118,"(500,10000)",gradvi_compound_lasso_init,gradvi_compound_lasso_init/equicorrgauss_118_l...
718,10,equicorrgauss,20,equicorrgauss/equicorrgauss_119,0.8,equicorrgauss/equicorrgauss_119,"(500,10000)",gradvi_compound_lasso_init,gradvi_compound_lasso_init/equicorrgauss_119_l...


In [15]:
dscout

,DSC,simulate,simulate.dims,simulate.se,simulate.sfix,simulate.pve,fit,fit.DSC_TIME,mse.err,coef_mse.err
0,1,equicorrgauss,"(500,10000)",3.518328,5,0.4,mr_ash,10.793000,13.792139,NaN
1,1,equicorrgauss,"(500,10000)",3.532144,10,0.4,mr_ash,10.543000,14.132777,NaN
2,1,equicorrgauss,"(500,10000)",4.439466,20,0.4,mr_ash,13.962000,24.171936,NaN
3,1,equicorrgauss,"(500,10000)",18.554388,250,0.4,mr_ash,17.545000,528.056334,NaN
4,1,equicorrgauss,"(500,10000)",1.933117,5,0.6,mr_ash,8.707000,3.800681,NaN
...,...,...,...,...,...,...,...,...,...,...
1435,10,equicorrgauss,"(500,10000)",12.678184,250,0.6,gradvi_compound_lasso_init,152.227589,NaN,0.021105
1436,10,equicorrgauss,"(500,10000)",0.899456,5,0.8,gradvi_compound_lasso_init,131.903669,NaN,0.000002
1437,10,equicorrgauss,"(500,10000)",2.668140,10,0.8,gradvi_compound_lasso_init,139.484373,NaN,0.000065
1438,10,equicorrgauss,"(500,10000)",2.396502,20,0.8,gradvi_compound_lasso_init,98.365349,NaN,0.000056


In [16]:
def stratify_dfcol(df, colname, value):
    #return pd_utils.select_dfrows(df, [f"$({colname}) == {value}"])
    return df.loc[df[colname] == value]

def stratify_dfcols(df, condition_list):
    for (colname, value) in condition_list:
        df = stratify_dfcol(df, colname, value)
    return df

def stratify_dfcols_in_list(df, colname, values):
    return df.loc[df[colname].isin(values)]

In [17]:
primary_keys     = ["simulate", "simulate.dims", "simulate.sfix", "simulate.pve", "fit"]
secondary_keys   = ["DSC"]
target_stats     = ["mse", "coef_mse", "niter", "elbo", "runtime_wall", "runtime_cpu", "runtime_dsc", "convergence"]
dataread_stats   = ["niter", "elbo", "runtime_cpu", "runtime_wall", "convergence"]
is_force_refresh = [False for x in target_stats]

In [18]:
unique_rows = []

# First we create the unique value of the primary key
for pkey_uniq_vals in itertools.product(*[df[key].unique() for key in primary_keys]):    
    
    # Corresponding to each unique row of primary keys,
    # we find the unique secondary keys.
    p_rows = stratify_dfcols(
        df, 
        [(key, val) for key, val in zip(primary_keys, pkey_uniq_vals)]
    )
    for skey_uniq_vals in itertools.product(*[p_rows[key].unique() for key in secondary_keys]):
        unique_rows.append(pkey_uniq_vals + skey_uniq_vals)

In [19]:
len(unique_rows)

720

In [20]:
resdict = {**{x: {} for x in primary_keys}, 
           **{x: {} for x in secondary_keys}, 
           **{x: {} for x in target_stats},
          }

resdict

{'simulate': {},
 'simulate.dims': {},
 'simulate.sfix': {},
 'simulate.pve': {},
 'fit': {},
 'DSC': {},
 'mse': {},
 'coef_mse': {},
 'niter': {},
 'elbo': {},
 'runtime_wall': {},
 'runtime_cpu': {},
 'runtime_dsc': {},
 'convergence': {}}

In [21]:
def read_statistic_from_dscout(colname, df, keys, vals):
    dfrows = stratify_dfcols(df, [(k, v) for k, v in zip(keys, vals)])
    stat   = dfrows[~dfrows[colname].isnull()][colname].unique()
    if len(stat) == 1:
        return stat[0]
    else:
        print (f"Error fetching value for {colname}")
        print (vals)
        return stat[0]


def get_output_filename_from_db(outdir, row):
    method = row['fit'].values[0]
    fprefix = row['fit.output.file'].values[0]
    ext = 'pkl' if method.startswith('gradvi') else 'rds'
    fname = os.path.join(outdir, f"{fprefix}.{ext}")
    return fname


def read_output_file(outdir, db, keys, vals):
    dfrow = stratify_dfcols(db, [(k, v) for k, v in zip(keys, vals)])
    '''
    must be a single row after selection
    '''
    assert(dfrow.shape[0] == 1)
    fname = get_output_filename_from_db(outdir, dfrow)
    data  = dsc_io.load_dsc(fname)
    return data


def get_niter_from_data(data):
    if 'fit' in data['model'].keys():
        if 'iter' in data['model']['fit'].keys():
            return data['model']['fit']['iter']
    if 'niter' in data['model'].keys():
        return data['model']['niter']
    
    
def get_runtime_from_data(data, rtype):
    rkey = f"runtime_{rtype}"
    if 'fit' in data['model'].keys():
        return data['model']['fit'][rkey]
    else:
        return data['model'][rkey]
    
    
def get_elbo_from_data(data):
    if 'fit' in data['model'].keys():
        return data['model']['fit']['varobj'][-1]
    else:
        return data['model']['elbo_path'][-1]
    
    
def get_convergence_from_data(data, mrash_maxiter = 2000):
    '''
    1: converged
    2: abnormal termination
    3: number of iterations reached limit
    4: unknown
    '''
    if 'fit' in data['model'].keys():
        niter = data['model']['fit']['iter']
        if niter == mrash_maxiter:
            conv_status = 3
        else:
            conv_status = 1
            
    else:
        success = data['model']['success']
        if success:
            conv_status = 1
        else:
            conv_status = 4
            #conv_message = data['model']['fitobj']['message']
            #if "ITERATIONS REACHED LIMIT" in conv_message:
            #    conv_status = 3
            #elif "ABNORMAL_TERMINATION_IN_LNSRCH" in conv_message:
            #    conv_status = 2
            #else:
            #    conv_status = 4
        
    return conv_status

    
def read_statistic(stat, data, msedf, keys, vals):
    if stat == 'mse':        
        return read_statistic_from_dscout('mse.err', msedf, keys, vals)
    
    elif stat == 'coef_mse':
        return read_statistic_from_dscout('coef_mse.err', msedf, keys, vals)
    
    elif stat == 'niter':
        return get_niter_from_data(data)
    
    elif stat == 'runtime_wall':
        return get_runtime_from_data(data, 'wall')
    
    elif stat == 'runtime_cpu':
        return get_runtime_from_data(data, 'cpu')
    
    elif stat == 'elbo':
        return get_elbo_from_data(data)
    
    elif stat == 'convergence':
        return get_convergence_from_data(data)
    
    elif stat == 'runtime_dsc':
        return read_statistic_from_dscout('fit.DSC_TIME', msedf, keys, vals)

In [22]:
if refresh_pickle:
    
    is_data_read_required = [True if x in dataread_stats else False for x in target_stats]
    
    for i, row in enumerate(unique_rows):
        for j, key in enumerate(primary_keys + secondary_keys):
            resdict[key][i] = row[j]

    for i, row in enumerate(unique_rows):
        is_stat_missing = [False if i in resdict[stat].keys() else True for stat in target_stats]
        data_read_flags = [ x & (y | z) for x, y, z in zip(is_data_read_required, is_stat_missing, is_force_refresh)]

        data = None
        if any(data_read_flags):
            data = read_output_file(dsc_output, df, primary_keys + secondary_keys, row)

        for j, stat in enumerate(target_stats):
            if is_stat_missing[j] or is_force_refresh[j]:
                val = read_statistic(stat, data, dscout, primary_keys + secondary_keys, row)
                resdict[stat][i] = val

    resdf = pd.DataFrame.from_dict(resdict)
    resdf.to_pickle(respkl)
    
else:
    resdf = pd.read_pickle(respkl)